<a href="https://colab.research.google.com/github/AmayFox/DataAnalysis/blob/main/yandex_bank_research/yandex_bank_binary_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## План исследования

1. [Шаг 1. Подготовка данных](#step1)


2. [Шаг 2. Исследование задачи](#step2)


3. [Шаг 3. Борьба с дисбалансом](#step3)


4. [Шаг 4. Тестирование модели](#step4)

# 1. Подготовка данных <a id="step1"></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import warnings
from sklearn.metrics import roc_auc_score
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/datasets/Churn.csv')
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [ ]:
df.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Скорее всего пропущенные значения в столбце Tenure это отсутсвие неджижимости. 

Заполним нулями пропущенные значения.

In [ ]:
df['Tenure'] = df['Tenure'].fillna(0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [ ]:
df['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

**Изменим типы данных таким образом**
+ RowNumber ✅
+ CustomerId ✅
+ Surname ✅
+ CreditScore ✅
+ Geography ➡️ Кодируем в 3 столбца: France Germany и Spain
+ Gender ➡️  Кодируем в 2 столбца: Male Female
+ Age ✅
+ Tenure ➡️  int64
+ Balance ✅
+ NumOfProducts ✅
+ HasCrCard ✅
+ IsActiveMember ✅ 
+ EstimatedSalary ✅
+ Exited ✅

In [ ]:
df['Tenure'] = df['Tenure'].astype(int)
df_ohe = pd.get_dummies(df, columns=['Geography','Gender'])
df_ohe.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,4,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [ ]:
target = df_ohe['Exited']
features = df_ohe.drop(['Exited', 'Surname', 'CustomerId'], axis=1)
print(target.shape, features.shape)

(10000,) (10000, 14)


In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target , test_size=0.20, random_state=12345, stratify = target)
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train , test_size=0.25, random_state=12345, stratify = target_train)

In [ ]:
print(features_train.shape, target_train.shape)
print(features_test.shape, target_test.shape)
print(features_valid.shape, target_valid.shape)

(6000, 14) (6000,)
(2000, 14) (2000,)
(2000, 14) (2000,)


# 2. Исследование задачи <a id="step2"></a>

In [ ]:
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Оставшихся клиентов значительно больше чем ушедших (соотношение 1:4).

Дисбаланс классов очевиден. Сначала исследуем задачу без учета дисбаланса.

In [ ]:
best_model = None
best_result = 0
est1, max_d = 0,0
for est in range(5, 30):
    for j in range(8,50):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = j)
        model.fit(features_train, target_train) 
        result = f1_score(target_valid, model.predict(features_valid))
        if result > best_result:
            best_model = model
            best_result = result
            est1,max_d = est, j
    print("F1-score:",best_result,"  Кол-во деревьев:",est1, "  Макс. грубина",max_d)
print("F1-score наилучшей модели случайного леса на валидационной выборке:", best_result)
print('Кол-во деревьев:', est1)
print('Максимальная глубина:',max_d)

F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол-во деревьев: 5   Макс. грубина 18
F1-score: 0.5875706214689266   Кол

Максимальное значение f1 меры без учета дисбаланса классов: 0.5875
    
Этого недостаточно чтобы считать исследование успешным. Проведем ребалансировку классов.

In [ ]:
auc_roc = roc_auc_score(target_valid, best_model.predict(features_valid))
print(auc_roc)

0.7263380483719467


# 3. Борьба с дисбалансом <a id="step3"></a>

Для начала попробуем просто сделать классы салансированными при помощи задания весов.

In [ ]:
best_model_2 = None
best_result_2 = 0
est2, max_d_2 = 0,0
for est in range(5, 30):
    for j in range(8,50):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = j,class_weight='balanced')
        model.fit(features_train, target_train) 
        result = f1_score(target_valid, model.predict(features_valid))
        if result > best_result_2:
            best_model_2 = model
            best_result_2 = result
            est2,max_d_2 = est, j
    print("F1-score:",best_result_2,"  Кол-во деревьев:",est2, "  Макс. грубина",max_d_2)
print("F1-score наилучшей модели случайного леса на валидационной выборке:", best_result_2)
print('Кол-во деревьев:', est2)
print('Максимальная глубина:',max_d_2)

F1-score: 0.5941240478781284   Кол-во деревьев: 5   Макс. грубина 8
F1-score: 0.5966480446927374   Кол-во деревьев: 6   Макс. грубина 9
F1-score: 0.6191489361702127   Кол-во деревьев: 7   Макс. грубина 8
F1-score: 0.6191489361702127   Кол-во деревьев: 7   Макс. грубина 8
F1-score: 0.6191489361702127   Кол-во деревьев: 7   Макс. грубина 8
F1-score: 0.6191489361702127   Кол-во деревьев: 7   Макс. грубина 8
F1-score: 0.6255506607929515   Кол-во деревьев: 11   Макс. грубина 9
F1-score: 0.6308724832214764   Кол-во деревьев: 12   Макс. грубина 9
F1-score: 0.6308724832214764   Кол-во деревьев: 12   Макс. грубина 9
F1-score: 0.6344206974128234   Кол-во деревьев: 14   Макс. грубина 9
F1-score: 0.6359550561797752   Кол-во деревьев: 15   Макс. грубина 9
F1-score: 0.6380090497737556   Кол-во деревьев: 16   Макс. грубина 9
F1-score: 0.6382022471910113   Кол-во деревьев: 17   Макс. грубина 9
F1-score: 0.6382022471910113   Кол-во деревьев: 17   Макс. грубина 9
F1-score: 0.6382022471910113   Кол-во де

Максимальное значение f1-меры с учетом дисбаланса классов: 0.64
    
Результаты допустимые но стоит попробовать другие методы.

In [ ]:
auc_roc = roc_auc_score(target_valid, best_model_2.predict(features_valid))
print(auc_roc)

0.7777901167731677


Площадь под графиком ROC увеличилась при увеличении значения f1-меры.

Для найденой лучшей модели выберем другой порог классификации.

In [ ]:
probabilities_valid = best_model_2.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
for threshold in np.arange(0.2, 0.8, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    print("Порог = {:.2f} | F1_score = {:.6f}".format(
        threshold, f1_score(target_valid, predicted_valid)))

Порог = 0.20 | F1_score = 0.454762
Порог = 0.22 | F1_score = 0.470960
Порог = 0.24 | F1_score = 0.487060
Порог = 0.26 | F1_score = 0.506294
Порог = 0.28 | F1_score = 0.520349
Порог = 0.30 | F1_score = 0.546863
Порог = 0.32 | F1_score = 0.562602
Порог = 0.34 | F1_score = 0.577740
Порог = 0.36 | F1_score = 0.597518
Порог = 0.38 | F1_score = 0.611524
Порог = 0.40 | F1_score = 0.606414
Порог = 0.42 | F1_score = 0.614913
Порог = 0.44 | F1_score = 0.626327
Порог = 0.46 | F1_score = 0.637969
Порог = 0.48 | F1_score = 0.639908
Порог = 0.50 | F1_score = 0.640288
Порог = 0.52 | F1_score = 0.626238
Порог = 0.54 | F1_score = 0.607692
Порог = 0.56 | F1_score = 0.599201
Порог = 0.58 | F1_score = 0.584958
Порог = 0.60 | F1_score = 0.560694
Порог = 0.62 | F1_score = 0.561194
Порог = 0.64 | F1_score = 0.537867
Порог = 0.66 | F1_score = 0.520900
Порог = 0.68 | F1_score = 0.500000
Порог = 0.70 | F1_score = 0.476684
Порог = 0.72 | F1_score = 0.439640
Порог = 0.74 | F1_score = 0.416357
Порог = 0.76 | F1_sc

Изменение порога плодов не дало, лучшая f1-мера при пороге 0.5 

Попробуем еще один метод борьбы с дисбалансом. Попробуем увеличить выборку.

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)
print(features_upsampled.shape, target_upsampled.shape)

(9669, 14) (9669,)


In [ ]:
best_model_3 = None
best_result_3 = 0
est3, max_d_3 = 0,0
for est in range(5, 30):
    for j in range(8,30):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = j)
        model.fit(features_upsampled, target_upsampled) 
        result = f1_score(target_valid, model.predict(features_valid))
        if result > best_result_3:
            best_model_3 = model
            best_result_3 = result
            est3,max_d_3 = est, j
    print("F1-score:",best_result_3,"  Кол-во деревьев:",est3, "  Макс. грубина",max_d_3)
print("F1-score наилучшей модели случайного леса на валидационной выборке:", best_result_3)
print('Кол-во деревьев:', est3)
print('Максимальная глубина:',max_d_3)

F1-score: 0.5826612903225807   Кол-во деревьев: 5   Макс. грубина 8
F1-score: 0.5845845845845845   Кол-во деревьев: 6   Макс. грубина 8
F1-score: 0.5900783289817233   Кол-во деревьев: 7   Макс. грубина 20
F1-score: 0.6024096385542169   Кол-во деревьев: 8   Макс. грубина 8
F1-score: 0.6055045871559632   Кол-во деревьев: 9   Макс. грубина 21
F1-score: 0.6075949367088607   Кол-во деревьев: 10   Макс. грубина 9
F1-score: 0.6130346232179226   Кол-во деревьев: 11   Макс. грубина 8
F1-score: 0.6137931034482759   Кол-во деревьев: 12   Макс. грубина 12
F1-score: 0.6202090592334495   Кол-во деревьев: 13   Макс. грубина 12
F1-score: 0.621676891615542   Кол-во деревьев: 14   Макс. грубина 8
F1-score: 0.6258823529411764   Кол-во деревьев: 15   Макс. грубина 12
F1-score: 0.6258823529411764   Кол-во деревьев: 15   Макс. грубина 12
F1-score: 0.6383981154299176   Кол-во деревьев: 17   Макс. грубина 12
F1-score: 0.6383981154299176   Кол-во деревьев: 17   Макс. грубина 12
F1-score: 0.6383981154299176   К

Увеличение выборки дало ~0.64 значения f1-меры. Оставим модель полученную при добавлении весов как лучшую.

In [ ]:
auc_roc = roc_auc_score(target_valid, best_model_3.predict(features_valid))
print(auc_roc)

0.7792515165396521


Значение площади практически то же самое что и в предыдущей модели.

# 4. Тестирование модели <a id="step4"></a>

In [ ]:
final_predict = best_model_2.predict(features_test)
print("Значение f1-меры для лучшей полученной модели на тестовой выборке:", f1_score(target_test, final_predict))

Значение f1-меры для лучшей полученной модели на тестовой выборке: 0.6239813736903376


# Чек-лист готовности проекта


- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*